In [ ]:
# Load libraries
import pandas as pd
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [ ]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/hepatitis/hepatitis.data"
names = ['sclass', 'age', 'sex', 'steroid', 'antivirals','fatigue','malaise',
         'anorexia','liver-big','liver-firm','spleen-palpable','spiders','ascites','varices',
         'bilirubin','alk-phosphate','sgot','alumin','protime','histology']
dataset = pd.read_csv(url, names=names)

In [ ]:
dataset.columns

In [ ]:

print(dataset.shape)
dataset.head()

In [ ]:

print(dataset.head(20))

In [ ]:
# 
print(dataset.describe())

In [ ]:
# class distribution
print(dataset.groupby('sclass').size())
print(dataset.groupby('age').size())
print(dataset.groupby('sex').size())
print(dataset.groupby('antivirals').size())
print(dataset.groupby('histology').size())


In [ ]:
# box and whisker plots
dataset.plot(kind='box' , subplots=True, layout=(2,3), sharex=False, sharey=False )
#dataset.plot(kind='box', subplots=True, layout=(1,5), sharex=False, sharey=False)
print("This gives us a much clearer idea of the distribution of the input attributes:")
plt.show()

In [ ]:
# histograms
dataset.hist()
print("It looks like perhaps two of the input variables have a Gaussian distribution \n")
print("This is useful to note as we can use algorithms that can exploit this assumption.")
plt.show()

In [ ]:
# scatter plot matrix
scatter_matrix(dataset)
print("Note the diagonal grouping of some pairs of attributes. \
This suggests a high correlation and a predictable relationship.") 
plt.show()

In [ ]:
labels = dataset.sclass.values
#print(labels)
features = dataset[['age', 'sex', 'steroid', 'antivirals','fatigue','malaise',
         'anorexia','liver-big','liver-firm','spleen-palpable','spiders','ascites','varices',
         'bilirubin','alk-phosphate','sgot','alumin','protime','histology']]
#print(features)

In [ ]:
features.head()

In [ ]:
pd.get_dummies(features).head()

In [ ]:
features_dummies = pd.get_dummies(features, columns=['steroid', 'fatigue','malaise',
         'anorexia','liver-big','liver-firm','spleen-palpable','spiders','ascites','varices',
         'bilirubin','alk-phosphate','sgot','alumin','protime'])
features_dummies.head(n=16)

In [ ]:
data = features_dummies.values

In [ ]:
import numpy as np
np.isnan(data).any()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer
# Use PreProcessing, when database is missing

#train_data, test_data, train_labels, test_labels = train_test_split(data, labels, random_state=0)

X_train, X_validation, Y_train, Y_validation = train_test_split(data, labels, random_state=0)

imp = Imputer()
#imp.fit(train_data)
imp.fit(X_train)
#train_data_finite = imp.transform(train_data)
#test_data_finite = imp.transform(test_data)
X_data_finite = imp.transform(X_train)
Y_data_finite = imp.transform(X_validation)

In [ ]:
from sklearn.dummy import DummyClassifier

clf = DummyClassifier('most_frequent')
#clf.fit(train_data_finite, train_labels)
clf.fit(X_data_finite, Y_train)
print("Prediction accuracy: %f" % clf.score(Y_data_finite, Y_validation))

#train_data_finite
#train_labels
#test_labels
#test_data_finite

In [ ]:
# Split-out validation dataset
#array = dataset.values
#X = array[:,1:21]
#Y = array[:,0]
#validation_size = 0.20
#seed = 7
#X_train, X_validation, Y_train, Y_validation = \
 #       model_selection.train_test_split(X, Y, test_size=validation_size, random_state=seed)

In [ ]:
# Test options and evaluation metric
seed = 7
scoring = 'accuracy'

In [ ]:
# Spot Check Algorithms
models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))

print(type(models)) 

# evaluate each model in turn
results = []
names = []
for name, model in models:
    kfold = model_selection.KFold(n_splits=10, random_state=seed)
    cv_results = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

In [ ]:
# Compare Algorithms
fig = plt.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

In [ ]:
# Make predictions on validation dataset
knn = KNeighborsClassifier()
knn.fit(X_train, Y_train)
predictions = knn.predict(X_validation)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))